In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import copy
import time

In [3]:
import utils
from models import ResNet

In [4]:
# not working

def show_images(image):
    image = image / 2 + 0.5
    npim = image.numpy()
    X = np.transpose(npim, [0, 2, 3, 1])
    plt.imshow(X)

In [5]:
data_loaders, test_loader = utils.get_data_loaders()

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [6]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

False

In [8]:
net = ResNet.ResNet50()

Freeze first couple layers to keep imagenet
Only finetune the later layers

In [9]:
#for i, child in enumerate(model_ft.children()):
 #   if i < 7:
  #      for param in child.parameters():
   #         param.requires_grad = False
    #else:
     #   break

In [10]:
#num_ftrs = net.fc.in_features
#model_ft.fc = nn.Linear(num_ftrs, 10)

net = net.to(device)

criterion = nn.CrossEntropyLoss()

In [11]:
optimizer = optim.Adam(net.parameters())
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=32)

Size of CIFAR-10

In [12]:
dataset_size = {'train': 4000,'val': 1000,'test': 1000}

In [26]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    
    start = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_accuracy = 0.0
    
    for epoch in range(num_epochs):
        
        print(str(epoch) + "/" + str(num_epochs))
        
        for phase in ['train', 'val']:
            
            print(phase)
            
            if phase == 'train':
                
                #scheduler.step()
                model.train()
                
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, targets in data_loaders[phase]:
                
                print(targets)
                
                inputs = inputs.to(device)
                targets = targets.to(device)

                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = outputs.max(1)
                    loss = criterion(outputs, targets)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == targets.data)
                
                epoch_loss = running_loss / dataset_size[phase]
                epoch_acc = running_corrects.double() / dataset_size[phase]
                
                print('Loss: ' + str(epoch_loss) + ", Epoch Accuracy: " + str(epoch_acc))
                
            if phase == 'val' and epoch_acc > best_accuracy:
                best_accuracy = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            
    train_time = time.time() - start
    
    print('Best Accuracy: ' + best_accuracy)
    
    model.load_state_dict(best_model_wts)
    return model

In [27]:
exp = 25
model_ft = train_model(net, criterion, optimizer, scheduler, exp)

0/25
train
tensor([1, 9, 6, 1, 3, 8, 3, 5, 4, 5])
Loss: 0.008244263529777527, Epoch Accuracy: tensor(0.0003, dtype=torch.float64)
tensor([4, 0, 7, 9, 8, 7, 5, 3, 3, 3])
Loss: 0.014124343991279603, Epoch Accuracy: tensor(0.0003, dtype=torch.float64)
tensor([6, 8, 7, 1, 0, 1, 8, 6, 1, 9])
Loss: 0.020309890508651732, Epoch Accuracy: tensor(0.0003, dtype=torch.float64)
tensor([4, 9, 9, 6, 3, 6, 7, 0, 9, 7])
Loss: 0.025568436980247498, Epoch Accuracy: tensor(0.0005, dtype=torch.float64)
tensor([1, 1, 0, 8, 7, 4, 4, 6, 9, 3])
Loss: 0.03280976057052612, Epoch Accuracy: tensor(0.0008, dtype=torch.float64)
tensor([2, 1, 5, 9, 6, 3, 0, 6, 2, 4])
Loss: 0.04103516161441803, Epoch Accuracy: tensor(0.0013, dtype=torch.float64)
tensor([0, 7, 8, 1, 0, 1, 5, 0, 1, 7])
Loss: 0.04630126893520355, Epoch Accuracy: tensor(0.0015, dtype=torch.float64)
tensor([1, 4, 7, 5, 8, 9, 5, 5, 0, 6])
Loss: 0.055753206610679625, Epoch Accuracy: tensor(0.0018, dtype=torch.float64)
tensor([4, 4, 7, 9, 6, 7, 7, 9, 7, 7])
L

Process Process-26:
Process Process-28:
Process Process-25:
Process Process-27:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  Fi

RuntimeError: DataLoader worker (pid 66196) exited unexpectedly with exit code 1. Details are lost due to multiprocessing. Rerunning with num_workers=0 may give better error trace.